## vgg 架构
VGG 的一个关键是使用很多有着相对小的 kernel（3\*3） 的卷积层然后接上一个池化层，之后再将这个模块重复多次。因此我们首先定义一个这样的块：

In [5]:
from mxnet.gluon import nn

def vgg_block(num_convs, channels):
    out = nn.Sequential()
    for _ in range(num_convs):
        out.add(
            nn.Conv2D(channels=channels, kernel_size=3,
                      padding=1, activation='relu')
        )
    out.add(nn.MaxPool2D(pool_size=2, strides=2))
    return out

定义将这些块进行连接

In [6]:
def vgg_stack(atchitecture):
    out = nn.Sequential()
    for (num_convs, channels) in architecture:
        out.add(vgg_block(num_convs, channels))
    return out

定义 VGG-16，它有 13个卷积层以及 3 个全连接层。

In [7]:
num_outputs = 10  # 最后输出的类别数量
architecture = ((2,64),(2, 128), (3,256),(3,512),(3,512))
net = nn.Sequential()
with net.name_scope():
    net.add(
        vgg_stack(architecture),
        nn.Flatten(),
        nn.Dense(4096, activation='relu'),
        nn.Dropout(0.5),
        nn.Dense(4096, activation='relu'),
        nn.Dropout(0.5),
        nn.Dense(num_outputs),
    )

## 模型训练
将图像扩大到 $96\times 96$ 大小进行训练

In [8]:
import sys
sys.path.append('..')
import utils
from mxnet import gluon
from mxnet import init

train_data, test_data = utils.load_data_fashion_mnist(
    batch_size=64, resize=96)

ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                       'sgd',
                       {learning_})

E:\APP\Anaconda3\envs\tensorflow1\lib\site-packages\mxnet\gluon\data\vision.py:126: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
E:\APP\Anaconda3\envs\tensorflow1\lib\site-packages\mxnet\gluon\data\vision.py:130: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


AttributeError: module 'mxnet.gluon.loss' has no attribute 'Softmax'